# How to deal with timestamps

## Install

In [2]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# !pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [3]:
import syft as sy
sy.requires(SYFT_VERSION)

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8.1b0 and the requirement <0.9


In [4]:
node = sy.orchestra.launch(name="pandas-test-domain-8", port=8080, reset=True)

Starting pandas-test-domain-8 server on 0.0.0.0:8080
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/0e22bbf87a1443ce96a65ded43560540.sqlite

Stopping process on port: 8080
Process 11362 terminated.


# Data owner: upload dataset

In [5]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

## Load data

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at /var/log/popularity-contest.

In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from syft.service.project.project import Project
from syft.util.util import autocache, PANDAS_DATA

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [9]:
popcon = pd.read_csv(autocache(f"{PANDAS_DATA}/popularity-contest"), sep=' ', )[:-1]
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

In [10]:
set(popcon["tag"])

{'<OLD>', '<RECENT-CTIME>', nan}

## Create mock data

Lets create a mock dataset

In [11]:
# weather_types = set(weather_2012_final.Weather.to_list())

In [12]:
from random import randint
import random

In [13]:
def get_random_timestamp():
    return "135" + "".join([str(randint(0,9)) for i in range(6)])

In [14]:
def get_mock_date(i):
    return str(parse('Jun 1 2010') + timedelta(days=i))

def get_mock_row(i):
    res = dict()
    for k, function in mock_functions.items():
        res[k] = function()
    return res
    

In [15]:
mock_functions = {'atime': lambda: get_random_timestamp(),
                  'ctime': lambda: get_random_timestamp(),
                  'package-name': lambda: random.choice([
                      'libghc-stm-dev',
                      'libqtdee-dev',
                      'geoclue-ubuntu-geoip',
                      'libdesktop-agnostic0',
                      'ubuntu-extras-keyring',
                      'libbsd0',
                      'libxres-dev']),
                  'mru-program': lambda: random.choice([
                      '/usr/bin/opam',
                      '/usr/bin/onboard',
                      '/lib/init/upstart-job',
                      '/usr/bin/page',
                  ]),
                  'tag': lambda: random.choice(['<OLD>', '<RECENT-CTIME>', 'nan'])}


In [16]:
mock = pd.DataFrame(data=[get_mock_row(i) for i in range(len(popcon))],
                    columns=popcon.columns)
    


Upload the data

In [17]:
dataset = sy.Dataset(name="test", asset_list=[sy.Asset(name=f"weather", data=popcon,
                                                       mock=mock, mock_is_real=False)])
root_domain_client.upload_dataset(dataset)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.65it/s]


Uploading: weather


<class 'syft.service.response.SyftSuccess'>: Dataset Added

In [18]:
popcon.head()

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,NaN
1,1387295796,1354370480,login,/bin/su,NaN
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


## Create user account

In [20]:
user = root_domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")
# todo: give user data scientist role
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: create syft_function

In [21]:
import numpy as np
import pandas as pd

## Summary

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

## Get mocks

In [22]:
ds = guest_domain_client.datasets[0]

In [23]:
asset = ds.assets[0]

In [24]:
popcon = asset.mock.syft_action_data

## Parsing Unix timestamps

The colums are the access time, created time, package name, recently used program, and a tag



In [25]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,135564910,135159468,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan
1,135156487,135255544,ubuntu-extras-keyring,/usr/bin/opam,<RECENT-CTIME>
2,135738541,135392007,libbsd0,/lib/init/upstart-job,nan
3,135527894,135383410,libxres-dev,/usr/bin/opam,<OLD>
4,135630303,135065212,libghc-stm-dev,/usr/bin/opam,<RECENT-CTIME>


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [26]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [27]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [28]:
popcon['atime'].dtype

dtype('<M8[ns]')

So now we can look at our `atime` and `ctime` as dates!

In [29]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1974-04-19 00:55:10,1974-04-14 08:17:48,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan
1,1974-04-14 07:28:07,1974-04-15 10:59:04,ubuntu-extras-keyring,/usr/bin/opam,<RECENT-CTIME>
2,1974-04-21 01:09:01,1974-04-17 00:53:27,libbsd0,/lib/init/upstart-job,nan
3,1974-04-18 14:38:14,1974-04-16 22:30:10,libxres-dev,/usr/bin/opam,<OLD>
4,1974-04-19 19:05:03,1974-04-13 06:06:52,libghc-stm-dev,/usr/bin/opam,<RECENT-CTIME>


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.


In [30]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [31]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [32]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
1653,1974-04-22 12:12:54,1974-04-24 01:22:20,ubuntu-extras-keyring,/lib/init/upstart-job,nan
2263,1974-04-18 09:03:07,1974-04-24 01:18:39,geoclue-ubuntu-geoip,/usr/bin/opam,nan
1534,1974-04-16 20:32:10,1974-04-24 01:15:29,ubuntu-extras-keyring,/usr/bin/opam,<OLD>
134,1974-04-16 10:34:35,1974-04-24 00:25:13,ubuntu-extras-keyring,/usr/bin/opam,<RECENT-CTIME>
1423,1974-04-15 17:49:41,1974-04-23 23:43:14,ubuntu-extras-keyring,/usr/bin/page,<RECENT-CTIME>
1248,1974-04-15 13:48:14,1974-04-23 23:17:53,geoclue-ubuntu-geoip,/usr/bin/page,nan
220,1974-04-21 17:29:07,1974-04-23 23:09:54,ubuntu-extras-keyring,/usr/bin/page,<OLD>
1692,1974-04-21 12:12:54,1974-04-23 22:54:45,ubuntu-extras-keyring,/usr/bin/page,<OLD>
727,1974-04-13 17:53:30,1974-04-23 22:27:20,geoclue-ubuntu-geoip,/usr/bin/opam,<RECENT-CTIME>
419,1974-04-13 18:13:56,1974-04-23 22:09:59,ubuntu-extras-keyring,/usr/bin/page,nan


Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

## Putting it together

Now we want to request the full code execution.

Let's put all that together, to prove how easy it is. 6 lines of magical pandas!

In [33]:
@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]

Request code execution

In [34]:
req = guest_domain_client.api.services.code.request_code_execution(find_recently_installed)

In [35]:
submitted_code = guest_domain_client.code[0]

In [36]:
assert guest_domain_client.api.services.code.get_all()

Create and submit project

In [37]:
new_project = sy.Project(name="Pandas Chapter 8",
                         description="Hi, I would like to get some insights about the installed programs")

In [38]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [39]:
guest_domain_client.submit_project(new_project)

<class 'syft.service.response.SyftSuccess'>: Project Submitted

# Data owner: execute syft_function

In [41]:
from syft import MessageStatus

In [43]:
domain_client = node.login(email="info@openmined.org", password="changethis")

# Get messages

In [44]:
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)

In [45]:
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,62cae0ff969141ed83117c402583d554,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 09:15:08,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,7fec517bec5d4f4196e4c0f01d65a2c5,Project Approval,MessageStatus.UNDELIVERED,2023-05-16 09:15:10,<<class 'syft.service.project.project.Project'...


In [46]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [47]:
request = project_message.link.requests[0]
func = request.changes[0].link
op = func.output_policy_type

In [48]:
func

```python
class UserCode:
  id: str = 0934806d72164d3eb9d9e537b2eed4ae
  node_uid: str = 0e22bbf87a1443ce96a65ded43560540
  user_verify_key: str = 1eeaaa21026d19bf2b23e6881ad92d1e5fbedfa9dca95605b38a0e4b4d47a74b
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]
"
  input_policy_type: str = <class 'syft.service.policy.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='pandas-test-domain-8', verify_key=d839a92ba1ddfe86df4215db8ad7d9ba7e88b3b7a8bfab6e916937253a6c941c): {'popcon': <UID: a379eda8c40c46f38b3641b9e1026fca>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.service.policy.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "def user_func_find_recently_installed_1eeaaa21026d19bf2b23e6881ad92d1e5fbedfa9dca95605b38a0e4b4d47a74b_76996a89952a0cd5f53e2e6da4ba9f2787211c4990d0fb1f69c5224b7af1b38c(popcon):

    def find_recently_installed(popcon):
        import pandas as pd
        popcon['atime'] = popcon['atime'].astype(int)
        popcon['ctime'] = popcon['ctime'].astype(int)
        popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
        popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
        popcon = popcon[popcon['atime'] > '1970-01-01']
        nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
        return nonlibraries.sort_values('ctime', ascending=False)[:10]
    result = find_recently_installed(popcon=popcon)
    return result"
  service_func_name: str = "find_recently_installed"
  unique_func_name: str = "user_func_find_recently_installed_1eeaaa21026d19bf2b23e6881ad92d1e5fbedfa9dca95605b38a0e4b4d47a74b_76996a89952a0cd5f53e2e6da4ba9f2787211c4990d0fb1f69c5224b7af1b38c"
  user_unique_func_name: str = "user_func_find_recently_installed_1eeaaa21026d19bf2b23e6881ad92d1e5fbedfa9dca95605b38a0e4b4d47a74b"
  code_hash: str = "76996a89952a0cd5f53e2e6da4ba9f2787211c4990d0fb1f69c5224b7af1b38c"
  signature: str = (popcon)
  status: str = {NodeView(node_name='pandas-test-domain-8', verify_key=d839a92ba1ddfe86df4215db8ad7d9ba7e88b3b7a8bfab6e916937253a6c941c): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['popcon']
  enclave_metadata: str = None

```

In [49]:
find_recently_installed = func.unsafe_function

In [50]:
real_data = domain_client.datasets[0].assets[0].data.syft_action_data

In [51]:
real_data

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,None
1,1387295796,1354370480,login,/bin/su,None
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,None
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,None
...,...,...,...,...,...
2892,0,0,libreadline-dev,<NOFILES>,None
2893,0,0,notify-osd-icons,<NOFILES>,None
2894,0,0,python-apt-common,<NOFILES>,None
2895,0,0,libindicator-messages-status-provider1,<NOFILES>,None


In [52]:
real_result = find_recently_installed(popcon=real_data)

In [53]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)

# Data Owner: fetch result

In [54]:
guest_client.api.services.code[0].status

{NodeView(node_name='pandas-test-domain-8', verify_key=d839a92ba1ddfe86df4215db8ad7d9ba7e88b3b7a8bfab6e916937253a6c941c): <UserCodeStatus.EXECUTE: 'execute'>}

In [55]:
asset = guest_client.datasets[0].assets[0]

In [56]:
real_result = guest_client.api.services.code.find_recently_installed(popcon=asset)

In [57]:
real_result

,atime,ctime,package-name,mru-program,tag
57,2013-12-17 04:55:39,2013-12-17 04:55:42,ddd,/usr/bin/ddd,<RECENT-CTIME>
450,2013-12-16 20:03:20,2013-12-16 20:05:13,nodejs,/usr/bin/npm,<RECENT-CTIME>
454,2013-12-16 20:03:20,2013-12-16 20:05:04,switchboard-plug-keyboard,/usr/lib/plugs/pantheon/keyboard/options.txt,<RECENT-CTIME>
445,2013-12-16 20:03:20,2013-12-16 20:05:04,thunderbird-locale-en,/usr/lib/thunderbird-addons/extensions/langpac...,<RECENT-CTIME>
396,2013-12-16 20:08:27,2013-12-16 20:05:03,software-center,/usr/sbin/update-software-center,<RECENT-CTIME>
449,2013-12-16 20:03:20,2013-12-16 20:05:00,samba-common-bin,/usr/bin/net.samba3,<RECENT-CTIME>
397,2013-12-16 20:08:25,2013-12-16 20:04:59,postgresql-client-9.1,/usr/lib/postgresql/9.1/bin/psql,<RECENT-CTIME>
398,2013-12-16 20:08:23,2013-12-16 20:04:58,postgresql-9.1,/usr/lib/postgresql/9.1/bin/postmaster,<RECENT-CTIME>
452,2013-12-16 20:03:20,2013-12-16 20:04:55,php5-dev,/usr/include/php5/main/snprintf.h,<RECENT-CTIME>
440,2013-12-16 20:03:20,2013-12-16 20:04:54,php-pear,/usr/share/php/XML/Util.php,<RECENT-CTIME>
